<a href="https://colab.research.google.com/github/tr-dev-bc/Modern_CV_Assignments/blob/main/midterm24_step_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# STEP 4

In [1]:
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt         # used to plot error later
# helps running in jupyter
%matplotlib inline

from torchvision import datasets

# Set device
device = "cuda" if torch.cuda.is_available() else "cpu"         # switch for gpu else cpu

# load and normalize the fmnist dataset
data_folder = '~/FMNIST'
fmnist = datasets.FashionMNIST(data_folder, download=True, train=True)
tr_images = fmnist.data
tr_targets = fmnist.targets

100%|██████████| 26.4M/26.4M [00:02<00:00, 12.7MB/s]


Extracting /root/FMNIST/FashionMNIST/raw/train-images-idx3-ubyte.gz to /root/FMNIST/FashionMNIST/raw



100%|██████████| 29.5k/29.5k [00:00<00:00, 202kB/s]


Extracting /root/FMNIST/FashionMNIST/raw/train-labels-idx1-ubyte.gz to /root/FMNIST/FashionMNIST/raw



100%|██████████| 4.42M/4.42M [00:01<00:00, 3.75MB/s]


Extracting /root/FMNIST/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to /root/FMNIST/FashionMNIST/raw



100%|██████████| 5.15k/5.15k [00:00<00:00, 3.91MB/s]

Extracting /root/FMNIST/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to /root/FMNIST/FashionMNIST/raw



In [2]:
class FMNISTDataset(Dataset):
    def __init__(self, x, y):
        x = x.float()/(255) # Normalizes images to [0,1]
        x = x.view(-1,1, 28,28)
        self.x, self.y = x, y
    def __getitem__(self, ix):
        x, y = self.x[ix], self.y[ix]  # unpack tuple into variables
        return x.to(device), y.to(device)
    def __len__(self):
        return len(self.x)

In [3]:
def get_data():
    train = FMNISTDataset(tr_images, tr_targets)
    trn_dl = DataLoader(train, batch_size=64, shuffle=True)         # batch size is 64
    return trn_dl
    #The DataLoader class in PyTorch loads dataset data, cuts it into batches, shuffles it

In [6]:
from torch.optim import SGD
def get_model():
  class neuralnet(nn.Module):
    def __init__(self):
      super().__init__()
      # A convolutional layer with 32 filters, kernel size of 3, and stride of 1
        self.conv_layer = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, stride=1)  # added to process images, 1 channel for grayscale
      # A max-pooling layer with a kernel size of 2
        self.pool = nn.MaxPool2d(kernel_size=2)  # downsampling feature maps
        self.flatten = nn.Flatten()  # convert feature map to single D tensor
      # Flatten layer to connect the convolutional layers to the fully connected layers
        self.input_to_hidden_layer = nn.Linear(32 * 13 * 13, 128)  # changed input size after conv and pool
        self.batch_norm = nn.BatchNorm1d(128)
        self.hidden_layer_activation = nn.ReLU()
        self.hidden_to_output_layer = nn.Linear(128, 10)
    def forward(self,x):
        x = self.conv_layer(x)
        x = self.pool(x)
        x = self.flatten(x)
        x = self.input_to_hidden_layer(x)
        x0 = self.batch_norm(x)
        x1 = self.hidden_layer_activation(x0)
        x2 = self.hidden_to_output_layer(x1)
          return x2
  model = neuralnet().to(device) # moves to the GPU to make it faster
  loss_fn = nn.CrossEntropyLoss()     # requ ired cross entropy loss function
  optimizer = SGD(model.parameters(), lr=1e-2) # model.parameters() retrieves all the parameters of the model that need to be optimized.
  return model, loss_fn, optimizer

IndentationError: unexpected indent (<ipython-input-6-a7fdce0f950d>, line 7)

In [ ]:
# loading up traiing data
train_loader = get_data()
model, loss_fn, optimizer = get_model()

# training loop with tracking of loss
num_epochs = 10
train_losses = []

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0

    for x, y in train_loader:
        x, y = x.to(device), y.to(device)

        # Forward pass
        optimizer.zero_grad()
        predictions = model(x)
        loss = loss_fn(predictions, y)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    average_loss = epoch_loss / len(train_loader)
    train_losses.append(average_loss)
    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {average_loss:.4f}')

PLOT OF LOSS:

In [ ]:
plt.plot(train_losses)
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training Loss')
plt.grid()
plt.show()

# STEP 4
